<a href="https://colab.research.google.com/github/luanps/pyserini/blob/master/Run_pyserini_tct_colbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# TCT Colbert Passage Ranking on MSMARCO

Sheng-Chieh Lin, Jheng-Hong Yang, and Jimmy Lin. Distilling Dense Representations for Ranking using Tightly-Coupled Teachers. arXiv:2010.11386, October 2020.


## Install dependencies

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
%%capture
!pip install pyserini

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
!pip install faiss-cpu

     |████████████████████████████████| 8.6 MB 4.5 MB/s 


## Dense retrieval with TCT-ColBERT, brute-force index

In [ ]:
!python -m pyserini.dsearch --topics msmarco-passage-dev-subset \
                             --index msmarco-passage-tct_colbert-bf \
                             --encoded-queries tct_colbert-msmarco-passage-dev-subset \
                             --batch-size 36 \
                             --threads 12 \
                             --output run.msmarco-passage.tct_colbert.bf.tsv \
                             --output-format msmarco

Using pre-defined topic order for msmarco-passage-dev-subset
Attempting to initialize pre-encoded queries tct_colbert-msmarco-passage-dev-subset.
/root/.cache/pyserini/queries/query-embedding-tct_colbert-msmarco-passage-dev-subset-20210419-9323ec.b2fe6494241639153f26cc61acf3b39d already exists, skipping download.
Initializing tct_colbert-msmarco-passage-dev-subset...
Attempting to initialize pre-built index msmarco-passage-tct_colbert-bf.
/root/.cache/pyserini/indexes/dindex-msmarco-passage-tct_colbert-bf-20210112-be7119.7312e0e7acec2a686e994902ca064fc5 already exists, skipping download.
Initializing msmarco-passage-tct_colbert-bf...
tcmalloc: large alloc 27162083328 bytes == 0x55deaf9b2000 @  0x7f1e52919887 0x7f1cc9410ce1 0x7f1cc94196b3 0x7f1cc92b7f90 0x55dea4ed9258 0x55dea500d18e 0x55dea50069ee 0x55dea4f99bda 0x55dea500bd00 0x55dea4f99afa 0x55dea5007c0d 0x55dea50069ee 0x55dea4f9a271 0x55dea4fdb159 0x55dea4fd80a4 0x55dea4f98c52 0x55dea500bc25 0x55dea4f99afa 0x55dea500bd00 0x55dea50069

In [ ]:
!gsutil cp run.* gs://luanps/information_retrieval/pyserini/tct_colbert_bf/

Copying file://run.msmarco-passage.tct_colbert.bf.tsv [Content-Type=text/tab-separated-values]...
|
Operation completed over 1 objects/127.0 MiB.                                    


### Evaluation

In [ ]:
#MRR Eval
!python -m pyserini.eval.msmarco_passage_eval msmarco-passage-dev-subset run.msmarco-passage.tct_colbert.bf.tsv >> mrr_eval.txt


In [ ]:
#TREC Eval
!python -m pyserini.eval.convert_msmarco_run_to_trec_run --input run.msmarco-passage.tct_colbert.bf.tsv \
                                                         --output run.msmarco-passage.tct_colbert.bf.trec

!python -m pyserini.eval.trec_eval -c -mrecall.1000 \
                                      -mmap msmarco-passage-dev-subset \
                                      run.msmarco-passage.tct_colbert.bf.trec >> trec_eval.txt

Done!


In [ ]:
!head run.msmarco-passage.tct_colbert.bf.trec

1048585 Q0 7187155 1 1.0 anserini
1048585 Q0 7187157 2 0.5 anserini
1048585 Q0 3856131 3 0.3333333333333333 anserini
1048585 Q0 7187158 4 0.25 anserini
1048585 Q0 7617411 5 0.2 anserini
1048585 Q0 7187160 6 0.16666666666666666 anserini
1048585 Q0 7617413 7 0.14285714285714285 anserini
1048585 Q0 7617404 8 0.125 anserini
1048585 Q0 7411281 9 0.1111111111111111 anserini
1048585 Q0 4072826 10 0.1 anserini


In [ ]:
!gsutil cp trec_eval.txt mrr_eval.txt gs://luanps/information_retrieval/pyserini/tct_colbert_bf/

Copying file://trec_eval.txt [Content-Type=text/plain]...
Copying file://mrr_eval.txt [Content-Type=text/plain]...
/ [2 files][  1.8 KiB/  1.8 KiB]                                                
Operation completed over 2 objects/1.8 KiB.                                      
